In [22]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    max_tokens=1000,
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "집에서 코딩을 할 때",
        "answer": """
            집에서 코딩을 할 때에는 다음과 같은 아이템을 추천합니다:
            -키보드 손목 받침대
            -좋은 의자 (패브릭 소재 편안한 의자)
            -좋은 키보드 (기계식 무소음 키보드)
            -좋은 마우스 (버티컬 마우스)
            -좋은 듀얼 모니터
            -좋은 컴퓨터 (1660 super 이상, i5 12600kf 이상)
        """
    },
    {
        "question": "카페에서 코딩을 할 때",
        "answer": """
            카페에서 코딩을 할 때에는 다음과 같은 아이템을 추천합니다:
            -에어팟 (소음 최소화)
            -맥북 (에어팟과 호환 가능, 코딩 할 때 쓰기 좋은 노트북)
            -무선 키보드 (가볍고 편한 무소음 키보드)
            -무선 마우스 (가볍고 조용한 무소음 축 마우스)
            -마우스 패드 (작고 휴대성이 좋은 마우스 패드)
        """
    },
    {
        "question": "수영장에 갈 때",
        "answer": """
            수영장에 갈 때에는 다음과 같은 아이템을 추천합니다:
            -수경 (수영을 위한 아이템)
            -수영모 (머리카락 빠짐 방지)
            -수영복 (수영을 위한 아이템)
        """
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question} 가지고 가면 좋은 아이템을 추천해줘.\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {purpose} 가지고 가면 좋은 아이템을 추천해줘.",
    input_variables=["purpose"]
)

chain = prompt | chat

prompt.format(purpose="차를 사러 갈 때")

TypeError: RandomExampleSelector.select_examples() takes 1 positional argument but 2 were given